In [2]:
! pip install weasyprint 

In [3]:
!pip install webrtcvad

     |████████████████████████████████| 66 kB 694 kB/s eta 0:00:011
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp36-cp36m-linux_x86_64.whl size=30065 sha256=5f5b68fbab13d078b4d65d4cce7a7998ab25cb17df17040dad33f777f433ba5e
  Stored in directory: /home/ec2-user/.cache/pip/wheels/ba/22/1c/d4e9707bbb27d469c384efc4263d8c7125219c1f088937289c
Successfully built webrtcvad


In [4]:
! pip install pydub

In [5]:
import webrtcvad
import collections
import contextlib
import sys
import wave
import numpy as np 
from pydub import AudioSegment
import pandas as pd
import os 
import cv2
from weasyprint import HTML
import time



/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [6]:
from sagemaker.pytorch.model import PyTorchModel 

In [7]:
from sagemaker import get_execution_role

In [8]:
role = get_execution_role()

pytorch_model = PyTorchModel(model_data='s3://sagemaker-us-east-2-808810818304/model/HPmodel_Serial.tar.gz', role=role,
                             framework_version='1.6.0',
                             py_version='py3',
                             entry_point='inference.py')



In [10]:
from sagemaker.deserializers import NumpyDeserializer
from sagemaker.serializers import NumpySerializer
predictor = pytorch_model.deploy(instance_type='ml.m4.xlarge',
                                     initial_instance_count=1,
                                     accelerator_type='ml.eia1.large' , 
                                Serializer = NumpySerializer() , 
                                Deserializer = NumpyDeserializer())

Defaulting to the only supported framework/algorithm version: 1.3.1. Ignoring framework/algorithm version: 1.6.0.


-----------------------------!

In [11]:
predictor.endpoint

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'pytorch-inference-eia-2021-04-16-00-15-00-416'

## Only test no valid 


In [12]:
import cv2
import json
image = cv2.imread('test_phs/Photo on 05-01-2021 at 6.42 AM #7.jpg')



In [13]:
def _npy_dumps(data):
    """
    Serializes a numpy array into a stream of npy-formatted bytes.
    """
    from six import BytesIO
    import numpy as np
    buffer = BytesIO()
    np.save(buffer, data)
    return buffer.getvalue()

def _npy_loads(data):
    """
    Deserializes npy-formatted bytes into a numpy array
    """
    from six import BytesIO
    import numpy as np
    stream = BytesIO(data)
    return np.load(stream)

In [14]:
'''
Prediction within notebook
'''
predictor.predict((image))

array([ -6.090973 , -40.535374 ,   0.8360443], dtype=float32)

In [15]:
import boto3

client = boto3.client('sagemaker-runtime')


In [18]:
response = client.invoke_endpoint(EndpointName=predictor.endpoint,
                                   ContentType='application/x-npy',
                                   Body=_npy_dumps(image))
for x in response['Body'].iter_lines() : 
    z = x
json.loads(z)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


[-6.090972900390625, -40.53537368774414, 0.8360443115234375]

In [19]:
def is_image (file_name) :
    supported_formats = ['png', 'jpg', 'jpeg' , 'JPG'] 
    for supported_format in supported_formats : 
        if supported_format in file_name :
            return True
    return False 

## Pose Cheating Detection 

In [20]:
def calc_pose_score(list_of_pose):
    score = []
    for i , x in enumerate(list_of_pose) : 
        if i == 0 :
            if x == 'center' : 
                score.append(0)
            else : 
                score.append(1)
        if i > 0 : 
            j = i - 1 
            if x == 'center' : 
                score.append(0)
            else : 
                if list_of_pose[j] == 'center':
                    score.append(1)
                if list_of_pose[j] == x:
                    score.append(1)
                if list_of_pose[j] != x and list_of_pose[j] != 'center' :
                    score.append(2)
    return score 

In [63]:
def get_pose_final_score (V_list_of_pose , H_list_of_pose , isreporting = False ) : 
    score_v = np.array(calc_pose_score(V_list_of_pose))
    score_h = np.array(calc_pose_score(H_list_of_pose))
    if isreporting :
        return score_v , score_h
    else : 
        return np.sum(score_v+ score_h)

In [64]:
def is_cheating_pose (V_list_of_pose , H_list_of_pose,threshold) : 
    cheating_score= get_pose_final_score(V_list_of_pose , H_list_of_pose)
    cheating_rate = cheating_score/ len(H_list_of_pose)
    cheating_result = cheating_rate > threshold 
    print('Cheating Score :',cheating_score)
    print('cheating rate :' ,cheating_rate  )
    return cheating_score ,cheating_rate ,  cheating_result

In [65]:
X = ['center','up','center','down','up']
J = ['left','right','center','center','right']
len(X),len(J)


(5, 5)

In [66]:
is_cheating_pose(X,J,1)

Cheating Score : 8
cheating rate : 1.6


(8, 1.6, True)

In [67]:
def transfer_to_directions(yaw , pitch ) :
    dir_yaw = '' 
    dir_pitch = '' 
    if -15 < yaw < 15 : 
        dir_yaw = 'center'
    elif yaw < -15 : 
        dir_yaw = 'right'
    elif yaw > 15 : 
        dir_yaw = 'left'
    if -15 < pitch < 15 : 
        dir_pitch = 'center'
    elif pitch < -15 : 
        dir_pitch = 'down'
    elif pitch > 15 : 
        dir_pitch = 'up'
    return dir_yaw , dir_pitch 




In [68]:
yaw , pitch = transfer_to_directions(-14 , 40 )
yaw , pitch

('center', 'up')

In [69]:
pitch_pose_list=[]
yaw_pose_list=[]

In [70]:
def is_cheating_pose_detection_run(path , threshold) : 
    pitch_pose_list=[]
    yaw_pose_list=[]
    count = 0
    for img_path in os.listdir(path) :
        if is_image (img_path) :
            count = count +1 
            frame = cv2.imread(path+'/'+img_path)
            pitch , yaw, _ = predictor.predict(frame)
            yaw , pitch = transfer_to_directions(yaw , pitch)
            pitch_pose_list.append(pitch)
            yaw_pose_list.append(yaw)
            print (count , 'Images Processed')
    print (yaw_pose_list , pitch_pose_list)
    return is_cheating_pose(pitch_pose_list,yaw_pose_list,threshold)

        

In [71]:
path= os.getcwd()+'/test_phs'
is_cheating_pose_detection_run(path,0.8)

1 Images Processed
2 Images Processed
3 Images Processed
4 Images Processed
5 Images Processed
6 Images Processed
7 Images Processed
8 Images Processed
['center', 'left', 'center', 'right', 'center', 'left', 'center', 'right'] ['down', 'center', 'center', 'center', 'down', 'center', 'up', 'center']
Cheating Score : 7
cheating rate : 0.875


(7, 0.875, True)

## VOICE ACTIVITY DETECTION

In [30]:
vad = webrtcvad.Vad(3)

In [31]:

class Frame(object):

    def __init__(self, bytes, timestamp, duration):
        self.bytes = bytes
        self.timestamp = timestamp
        self.duration = duration


def frame_generator(frame_duration_ms, audio, sample_rate):

    n = int(sample_rate * (frame_duration_ms / 1000.0) * 2)
    offset = 0
    timestamp = 0.0
    duration = (float(n) / sample_rate) / 2.0
    while offset + n < len(audio):
        yield Frame(audio[offset:offset + n], timestamp, duration)
        timestamp += duration
        offset += n

def read_wave(path):

    with contextlib.closing(wave.open(path, 'rb')) as wf:
        num_channels = wf.getnchannels()
        assert num_channels == 1
        sample_width = wf.getsampwidth()
        assert sample_width == 2
        sample_rate = wf.getframerate()
        assert sample_rate in (8000, 16000, 32000, 48000)
        pcm_data = wf.readframes(wf.getnframes())
        return pcm_data, sample_rate

In [32]:
def VAD_detection(wav_file_path,threshold) : 
    sound = AudioSegment.from_wav(wav_file_path)
    sound = sound.set_frame_rate(48000)
    sound = sound.set_channels(1)
    sound = sound.set_sample_width(2)
    sound.export(wav_file_path, format="wav")
    audio, sample_rate = read_wave(wav_file_path)
    frames = frame_generator(10, audio, sample_rate)
    frames = list(frames)
    is_speech=[]
    for frame in frames:
          is_speech.append(vad.is_speech(frame.bytes, sample_rate))
    is_speech_np = np.array(is_speech)
    return np.sum(is_speech) , (is_speech_np.sum()>threshold)

In [33]:
wav_file_path = 'student1/stud_1.wav'
threshold = 200 
VAD_detection(wav_file_path,threshold)

(302, True)

## Object Detection -YOLO-

In [34]:
def detect(imagePath, yoloPath, confidenceNeeded=0.5, threshold=0.3):

    # load the COCO class labels our YOLO model was trained on
    labelsPath = os.path.sep.join([yoloPath, "coco.names"])
    LABELS = open(labelsPath).read().strip().split("\n")

    # initialize a list of colors to represent each possible class label
    np.random.seed(42)
    COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
        dtype="uint8")

    # derive the paths to the YOLO weights and model configuration
    weightsPath = os.path.sep.join([yoloPath, "yolov3.weights"])
    configPath = os.path.sep.join([yoloPath, "yolov3.cfg"])

    # load our YOLO object detector trained on COCO dataset (80 classes)
    print("[INFO] loading YOLO from disk...")
    net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

    # load our input image and grab its spatial dimensions
    image = cv2.imread(imagePath)
    (H, W) = image.shape[:2]

    # determine only the *output* layer names that we need from YOLO
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    # construct a blob from the input image and then perform a forward
    # pass of the YOLO object detector, giving us our bounding boxes and
    # associated probabilities
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),
        swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(ln)
    end = time.time()

    # show timing information on YOLO
    print("[INFO] YOLO took {:.6f} seconds".format(end - start))

    # initialize our lists of detected bounding boxes, confidences, and
    # class IDs, respectively
    boxes = []
    confidences = []
    classIDs = []

    # loop over each of the layer outputs
    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            # extract the class ID and confidence (i.e., probability) of
            # the current object detection
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # filter out weak predictions by ensuring the detected
            # probability is greater than the minimum probability
            if confidence > confidenceNeeded:
                # scale the bounding box coordinates back relative to the
                # size of the image, keeping in mind that YOLO actually
                # returns the center (x, y)-coordinates of the bounding
                # box followed by the boxes' width and height
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                # use the center (x, y)-coordinates to derive the top and
                # and left corner of the bounding box
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                # update our list of bounding box coordinates, confidences,
                # and class IDs
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    # apply non-maxima suppression to suppress weak, overlapping bounding
    # boxes
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, confidenceNeeded,
        threshold)

    # ensure at least one detection exists
    result=[]
    if len(idxs) > 0:
        for i in idxs.flatten():
            result.append([LABELS[classIDs[i]], confidences[i]])

            

    # return the output
    return result


In [35]:
def get_palgarism_objects_txt(all_objects_that_can_be_detected , mode ):
    f = open(all_objects_that_can_be_detected, "r")
    list_of_palgarism_objects = [] 
    objects = f.read().split("\n") 
    for obj in objects : 
        if "*" in obj :
            list_of_palgarism_objects.append((''.join([i for i in obj if not i.isdigit()])).replace('*','').replace('-',''))
    if mode == 1 : 
        f = open("palgarism_objects_to_detect.txt", "w")
        for obj in list_of_palgarism_objects : 
            f.write(obj + "\n")
        f.close()
    if mode == 2 : 
        return list_of_palgarism_objects
    else : 
        return -1 
    
    
    


In [36]:
def filter_output_yolo(yolo_output , palgarism_objects) : 
    yolo_filtered_output = [] 
    for obj , conf in yolo_output : 
        if obj in palgarism_objects : 
            yolo_filtered_output.append([obj ,conf ])
    return yolo_filtered_output 

In [37]:
get_palgarism_objects_txt("palgarism_objects.txt",2)

['backpack',
 'handbag',
 'suitcase',
 'tvmonitor',
 'laptop',
 'cell phone',
 'book']

In [38]:
def run_yolo(image_path) : 
    output=detect(image_path,os.getcwd())
    yolo_filtered = filter_output_yolo(output ,
                        get_palgarism_objects_txt("palgarism_objects.txt",2))
    return yolo_filtered

    

In [41]:
def is_cheating_yolo(image_path) :
    cheating_objs = []
    yolo_out = run_yolo(image_path)
    if len(yolo_out) > 0 :
        for item in yolo_out : 
            cheating_objs.append(item[0])  
        return cheating_objs , True 
    else:
        return 'None' , False 
    

In [42]:
path_image = "test_4.jpeg" 
yolo_obj_list ,yolo_obj_result  =  is_cheating_yolo(path_image)

[INFO] loading YOLO from disk...
[INFO] YOLO took 1.078174 seconds


In [43]:
yolo_obj_list ,yolo_obj_result 


(['cell phone'], True)

### Integration

In [1]:
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]



In [44]:
def is_cheating_pose_yolo_detection_run(path , threshold) : 
    pitch_pose_list=[]
    yaw_pose_list=[]
    all_yolo_obj_list=[]
    all_yolo_obj_result=[]
    count = 0
    #for img_path in os.listdir(path) old not sorted:
    for img_path in sorted(os.listdir(path) , key=natural_keys):
        if is_image (img_path) :
            count = count +1 
            frame = cv2.imread(path+'/'+img_path)
            pitch , yaw, _ = predictor.predict(frame)
            yaw , pitch = transfer_to_directions(yaw , pitch)
            pitch_pose_list.append(pitch)
            yaw_pose_list.append(yaw)
            yolo_obj_list ,yolo_obj_result  = is_cheating_yolo(path+'/'+img_path)
            all_yolo_obj_list.append(yolo_obj_list)
            all_yolo_obj_result .append(yolo_obj_result)
            print (count , 'Images Processed')
    print (yaw_pose_list , pitch_pose_list)
    cheating_score ,cheating_rate ,  cheating_result = is_cheating_pose(pitch_pose_list,yaw_pose_list,threshold)
    return cheating_score ,cheating_rate ,  cheating_result ,all_yolo_obj_list,all_yolo_obj_result

In [45]:
def cheating_detection_complete_run(path , score_threshold , sound_threshold) : 
    detected_frames_list = []
    sound_result_list = [] 
    cheating_score ,cheating_rate ,  cheating_result ,all_yolo_obj_list,all_yolo_obj_result = is_cheating_pose_yolo_detection_run(path , score_threshold)
    for file in os.listdir(path) : 
        if 'wav' in file : 
            wav_file_path = path+'/'+file
            detected_frames , sound_result = VAD_detection(wav_file_path,sound_threshold)
            detected_frames_list.append(detected_frames)
            sound_result_list.append(sound_result)
    return cheating_score ,cheating_rate ,cheating_result ,all_yolo_obj_list,np.sum(all_yolo_obj_result) ,detected_frames_list,sound_result_list 
        
    

In [46]:
cheating_detection_complete_run(path,0.8,200)

[INFO] loading YOLO from disk...
[INFO] YOLO took 0.974235 seconds
1 Images Processed
[INFO] loading YOLO from disk...
[INFO] YOLO took 0.963979 seconds
2 Images Processed
[INFO] loading YOLO from disk...
[INFO] YOLO took 0.957979 seconds
3 Images Processed
[INFO] loading YOLO from disk...
[INFO] YOLO took 0.959109 seconds
4 Images Processed
[INFO] loading YOLO from disk...
[INFO] YOLO took 0.965252 seconds
5 Images Processed
[INFO] loading YOLO from disk...
[INFO] YOLO took 0.967860 seconds
6 Images Processed
[INFO] loading YOLO from disk...
[INFO] YOLO took 0.967581 seconds
7 Images Processed
[INFO] loading YOLO from disk...
[INFO] YOLO took 0.963053 seconds
8 Images Processed
['center', 'left', 'center', 'right', 'center', 'left', 'center', 'right'] ['down', 'center', 'center', 'center', 'down', 'center', 'up', 'center']
Cheating Score : 7
cheating rate : 0.875


(7,
 0.875,
 True,
 [['cell phone'], 'None', 'None', 'None', 'None', 'None', 'None', 'None'],
 1,
 [327],
 [True])

## Integration & Reporting

In [ ]:
from jinja2 import Environment, FileSystemLoader
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template("report.html")


In [ ]:
def cheating_detection_reporting_and_responsing(path,score_threshold,frames_threshold): 
    cheating_score ,cheating_rate ,cheating_result ,all_yolo_obj_list,all_yolo_obj_result ,detected_frames_list\
    ,sound_result_list = cheating_detection_complete_run(path,score_threshold,frames_threshold)
    report_df = pd.DataFrame ({
        'cheating score':cheating_score ,
        'cheating rate':cheating_rate ,
        'Pose cheating result' : cheating_result ,
        'objects_detected' :str(all_yolo_obj_list),
        'Num objects detected' : all_yolo_obj_result,
        'Num Frames of Human sound':detected_frames_list,
        'Sound Detection result':sound_result_list },index = ['Student']).T
    template_vars = {"title" : "Final Report",
                     "table": report_df.to_html()}
    html_out = template.render(template_vars)
    HTML(string=html_out).write_pdf("report.pdf")
    if cheating_result or all_yolo_obj_result or np.array(sound_result_list).sum() : 
        return True
    else :
        return False

In [ ]:
str([['cell phone'], 'None', 'None', 'None', 'None', 'None', 'None', 'None'])

In [ ]:
cheating_detection_reporting_and_responsing(path,0.8,200)